In [ ]:
!pip install --upgrade pip
!pip install opencv-python-headless Pillow numpy matplotlib scikit-image scikit-learn seaborn basicsr facexlib gfpgan realesrgan
!pip install pydicom nibabel medpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 74.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 131.0 MB/s  0:00:00
  DEPRECATION: Building 'basicsr' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'basicsr'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214817 sha256=cb0a486a6651606a446e628a07180998829ce843864cacbb12aa65679c9a9465
  Stored in 

In [ ]:
import torch
import torchvision
print(f"torch version: {torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")
print(f"cuda available: {torch.cuda.is_available()}")


torch version: 2.8.0+cu126
torchvision version: 0.23.0+cu126
cuda available: True


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!mkdir -p datasets/ham10000/
!unzip /content/drive/MyDrive/HAM10000/archive.zip -d datasets/ham10000/


Streaming output truncated to the last 5000 lines.
  inflating: datasets/ham10000/ham10000_images_part_2/ISIC_0029325.jpg  
  inflating: datasets/ham10000/ham10000_images_part_2/ISIC_0029326.jpg  
  inflating: datasets/ham10000/ham10000_images_part_2/ISIC_0029327.jpg  
  inflating: datasets/ham10000/ham10000_images_part_2/ISIC_0029328.jpg  
  inflating: datasets/ham10000/ham10000_images_part_2/ISIC_0029329.jpg  
  inflating: datasets/ham10000/ham10000_images_part_2/ISIC_0029330.jpg  
  inflating: datasets/ham10000/ham10000_images_part_2/ISIC_0029331.jpg  
  inflating: datasets/ham10000/ham10000_images_part_2/ISIC_0029332.jpg  
  inflating: datasets/ham10000/ham10000_images_part_2/ISIC_0029333.jpg  
  inflating: datasets/ham10000/ham10000_images_part_2/ISIC_0029334.jpg  
  inflating: datasets/ham10000/ham10000_images_part_2/ISIC_0029335.jpg  
  inflating: datasets/ham10000/ham10000_images_part_2/ISIC_0029336.jpg  
  inflating: datasets/ham10000/ham10000_images_part_2/ISIC_0029337.jpg  


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings('ignore')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [ ]:
class MedicalImageDataset(Dataset):
    def __init__(self, data_dir, transform=None, image_size=(224, 224)):
        self.data_dir = data_dir
        self.transform = transform
        self.image_size = image_size
        self.classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.images = []
        self.labels = []
        for class_name in self.classes:
            class_dir = os.path.join(data_dir, class_name)
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.images.append(os.path.join(class_dir, img_name))
                    self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]
        image = cv2.imread(image_path)
        if image is None:
            image = np.zeros((64, 64, 3), dtype=np.uint8)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, self.image_size)
        image = Image.fromarray(image)
        if self.transform:
            image = self.transform(image)
        return image, label

# Define transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
class MedicalCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(MedicalCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 14 * 14, 256), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = MedicalCNN(num_classes=2).to(device)
print(f"Model initialized on {device}")


Model initialized on cuda


In [ ]:
!ls -R datasets/ham10000/


Streaming output truncated to the last 5000 lines.
ISIC_0024320.jpg  ISIC_0025570.jpg  ISIC_0026820.jpg  ISIC_0028070.jpg
ISIC_0024321.jpg  ISIC_0025571.jpg  ISIC_0026821.jpg  ISIC_0028071.jpg
ISIC_0024322.jpg  ISIC_0025572.jpg  ISIC_0026822.jpg  ISIC_0028072.jpg
ISIC_0024323.jpg  ISIC_0025573.jpg  ISIC_0026823.jpg  ISIC_0028073.jpg
ISIC_0024324.jpg  ISIC_0025574.jpg  ISIC_0026824.jpg  ISIC_0028074.jpg
ISIC_0024325.jpg  ISIC_0025575.jpg  ISIC_0026825.jpg  ISIC_0028075.jpg
ISIC_0024326.jpg  ISIC_0025576.jpg  ISIC_0026826.jpg  ISIC_0028076.jpg
ISIC_0024327.jpg  ISIC_0025577.jpg  ISIC_0026827.jpg  ISIC_0028077.jpg
ISIC_0024328.jpg  ISIC_0025578.jpg  ISIC_0026828.jpg  ISIC_0028078.jpg
ISIC_0024329.jpg  ISIC_0025579.jpg  ISIC_0026829.jpg  ISIC_0028079.jpg
ISIC_0024330.jpg  ISIC_0025580.jpg  ISIC_0026830.jpg  ISIC_0028080.jpg
ISIC_0024331.jpg  ISIC_0025581.jpg  ISIC_0026831.jpg  ISIC_0028081.jpg
ISIC_0024332.jpg  ISIC_0025582.jpg  ISIC_0026832.jpg  ISIC_0028082.jpg
ISIC_0024333.jpg  ISIC_002

In [ ]:
# Change subfolder names as appropriate for your dataset
train_dataset = MedicalImageDataset('datasets/ham10000/', transform=train_transform)
test_dataset = MedicalImageDataset('datasets/ham10000/', transform=test_transform)
train_loader  = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader   = DataLoader(test_dataset, batch_size=16)
print(f"Training set size: {len(train_dataset)}")
print(f"Testing set size: {len(test_dataset)}")


Training set size: 20030
Testing set size: 20030


In [ ]:
import os
data_dir = 'datasets/ham10000/'  # or your actual dataset path
classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
print(f"Found classes: {classes}")
print(f"Number of classes: {len(classes)}")


Found classes: ['ham10000_images_part_2', 'HAM10000_images_part_2', 'ham10000_images_part_1', 'HAM10000_images_part_1']
Number of classes: 4


In [ ]:
num_classes = 4  # update to match your dataset classes

class MedicalCNN(nn.Module):
    def __init__(self, num_classes=num_classes):
        super(MedicalCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 14 * 14, 256), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = MedicalCNN(num_classes=num_classes).to(device)
print(f"Model initialized with {num_classes} classes on {device}")


Model initialized with 4 classes on cuda


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct_preds / total_preds
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}, Accuracy = {epoch_acc:.4f}")


Epoch 1: Loss = 1.3721, Accuracy = 0.2867
Epoch 2: Loss = 1.3575, Accuracy = 0.3039
Epoch 3: Loss = 1.3422, Accuracy = 0.3083
Epoch 4: Loss = 1.3268, Accuracy = 0.3145
Epoch 5: Loss = 1.3201, Accuracy = 0.3147
Epoch 6: Loss = 1.3073, Accuracy = 0.3164
Epoch 7: Loss = 1.2968, Accuracy = 0.3228
Epoch 8: Loss = 1.2860, Accuracy = 0.3257
Epoch 9: Loss = 1.2770, Accuracy = 0.3313
Epoch 10: Loss = 1.2642, Accuracy = 0.3305


In [ ]:
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

from sklearn.metrics import classification_report, accuracy_score

test_accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {test_accuracy:.4f}")
print("Classification Report:\n", classification_report(all_labels, all_preds))


Test Accuracy: 0.3424
Classification Report:
               precision    recall  f1-score   support

           0       0.45      0.10      0.17      5015
           1       0.46      0.31      0.37      5015
           2       0.27      0.34      0.30      5000
           3       0.33      0.63      0.44      5000

    accuracy                           0.34     20030
   macro avg       0.38      0.34      0.32     20030
weighted avg       0.38      0.34      0.32     20030

